In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
from datetime import datetime
import json

data can be downloaded from: https://webrobots.io/kickstarter-datasets/

In [2]:
data = pd.read_csv('data/kick_train.csv', index_col='id')
#data.shape #---> (114933, 30)
pd.set_option('display.max_columns', 30) 
data.head()

,Unnamed: 0,photo,name,blurb,goal,state,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,backers_count,static_usd_rate,creator,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
663816109,0,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Angular - Where Modern Art meets Cards,Angular is a minimalist card design for simpli...,17380.0,failed,angular-where-modern-art-meets-cards,False,US,USD,$,True,1459283229,1459283230,1455845363,1456694829,False,11,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1462931821,1,"{""small"":""https://ksr-ugc.imgix.net/assets/014...",Ladybeard is KAWAII-CORE,Original songs and music videos to jump start ...,24000.0,failed,ladybeard-is-kawaii-core,False,US,USD,$,True,1484110800,1484110800,1475568868,1480946454,False,304,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""JP"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1724358498,2,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Vegan Cafe Delivery Service in Vancouver BC,Our project is to launch a vegan lunch deliver...,40000.0,failed,vegancafeca,False,CA,CAD,$,True,1408549628,1408549628,1405218883,1405957628,False,2,0.926746,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""CA"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
314918941,3,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Photoetched Rail Yard Exposition,I have developed a process of my own which tra...,1000.0,successful,photoetched-rail-yard-exposition,False,US,USD,$,True,1364084914,1364084914,1360627778,1361496514,False,18,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1766165140,4,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Cinnamon Fletcher needs to be brought to life!,Need to pay an illustrator to bring my childre...,700.0,failed,cinnamon-fletcher-needs-to-be-brought-to-life,False,GB,GBP,Â£,False,1382600001,1382600002,1379704502,1380008001,False,0,1.602384,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [3]:
def data_prep(data):
    
    df =data.copy()
    #Changing all currencies in to USD 
    df['goal'] = (df['goal']*df['static_usd_rate']).astype(float).round(2)

    #extrating the categories from category column in "slug" key
    #df.category.replace({r'(.*slug":"([a-z]+\s?&?\s?[a-z]+)\/.*)' : r'\2'}, inplace=True, regex=True) #main categories 14
    df.category.replace({r'(.*slug\":\"(.*)\/([0-9a-z\"\'\"-]+\s?&?\s?[0-9a-z]+)\"\})' : r'\3'}, inplace=True, regex=True) #sub categories 126
    
    #extrating the locations, below will catch non asci sames like SÃ£o Paolo which I found in the database too)
    df.location.replace({r'(.*displayable_name":"((?:.*\s*),*\s*[A-Za-z ]*,*\s*[A-Za-z.*(*)*]+)","short.*)' : r'\2'}, inplace=True, regex=True)
    
    #extrating user id
    df.creator.replace({r'(.*id":([0-9]+),"avatar.*)' : r'\2'}, inplace=True, regex=True)
    
    #dropping unneccessary columns:
    df.drop(['Unnamed: 0', 'photo', 'slug', 'disable_communication', 'currency',
             'staff_pick', 'backers_count', 'currency_symbol', 'currency_trailing_code',
             'static_usd_rate', 'friends', 'is_starred', 'is_backing', 'permissions',
             'profile', 'urls', 'source_url'], axis=1, inplace=True)
    
    return df

In [48]:
#testing to see if any of the previously json columns did not get cought by above regexes:

def test_jsons(df):
    dff = df[['location', 'category', 'creator']]
    for col in dff:
        print(dff[dff[col].astype(str).str.contains(':')])

#test_jsons(preproc_data)     #all good!

In [7]:
preproc_data = data_prep(data)
print(preproc_data.shape)

(114933, 12)


In [8]:
pd.set_option('display.max_columns', 12)
#preproc_data.set_index('Unnamed: 0', inplace=True)
#preproc_data.reset_index(inplace=True)
preproc_data.head()

,name,blurb,goal,state,country,deadline,state_changed_at,created_at,launched_at,creator,location,category
id,,,,,,,,,,,,
663816109,Angular - Where Modern Art meets Cards,Angular is a minimalist card design for simpli...,17380.00,failed,US,1459283229,1459283230,1455845363,1456694829,200609383,"Raleigh, NC",playing cards
1462931821,Ladybeard is KAWAII-CORE,Original songs and music videos to jump start ...,24000.00,failed,US,1484110800,1484110800,1475568868,1480946454,1179655388,"Tokyo, Japan",pop
1724358498,Vegan Cafe Delivery Service in Vancouver BC,Our project is to launch a vegan lunch deliver...,37069.85,failed,CA,1408549628,1408549628,1405218883,1405957628,2078972315,"Vancouver, Canada",vegan
314918941,Photoetched Rail Yard Exposition,I have developed a process of my own which tra...,1000.00,successful,US,1364084914,1364084914,1360627778,1361496514,863236098,"Albuquerque, NM",mixed media
1766165140,Cinnamon Fletcher needs to be brought to life!,Need to pay an illustrator to bring my childre...,1121.67,failed,GB,1382600001,1382600002,1379704502,1380008001,1226281117,"Leicester, UK",children's books


In [9]:
#function to extract the datetime columns
unixtimes = lambda x: datetime.fromtimestamp(int(x)).strftime('%d/%m/%y')

datecols = preproc_data[['deadline', 'state_changed_at', 'created_at', 'launched_at']]

for col in datecols:
    datecols[col] = datecols[col].apply(unixtimes)

datecols.head()

C:\Users\ndadkhah\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,deadline,state_changed_at,created_at,launched_at
id,,,,
663816109,29/03/16,29/03/16,19/02/16,28/02/16
1462931821,11/01/17,11/01/17,04/10/16,05/12/16
1724358498,20/08/14,20/08/14,13/07/14,21/07/14
314918941,24/03/13,24/03/13,12/02/13,22/02/13
1766165140,24/10/13,24/10/13,20/09/13,24/09/13


In [10]:
todatetime= lambda x: pd.to_datetime(x, dayfirst=True, format='%d/%m/%y')
datecols = datecols.apply(todatetime)
#datecols.head()

In [11]:
timestamp = lambda x: pd.Timestamp(x, day=True)

In [12]:
#creating a pre_launch column: delta of 'launched_at' - 'created_at' which shows how long the project was live for
datecols['pre_launch'] = datecols['launched_at'].apply(timestamp) - datecols['created_at'].apply(timestamp)

In [13]:
#creating a post_launch column: delta of 'deadline' - 'launch_at' which shows how long the project was live for
datecols['post_launch'] = datecols['deadline'].apply(timestamp) - datecols['launched_at'].apply(timestamp)

In [14]:
dcols = ['pre_launch', 'post_launch']
datecols = datecols[dcols]
datecols.head()

,pre_launch,post_launch
id,,
663816109,9 days,30 days
1462931821,62 days,37 days
1724358498,8 days,30 days
314918941,10 days,30 days
1766165140,4 days,30 days


In [15]:
#as per above anallysys, the changes in the state_change and dealine if minus then not succesful and if more than almost 2 mins
#also most likely not succesful, so in order to have this info in less than day measures I am just deducting the unixtimes and
#using them, using this column did actually change the accuracy of the models. 
preproc_data['time_to_state_change'] = preproc_data['state_changed_at'] - preproc_data['deadline']

In [58]:
#these extra columns did not increase the accuracy of the models, so not using them

#unixtimes2 = lambda x: datetime.fromtimestamp(int(x)).strftime('%B')

#preproc_data['launch_month'] = preproc_data['launched_at'].apply(unixtimes2)
#preproc_data['deadline_month'] = preproc_data['deadline'].apply(unixtimes2)

In [16]:
preproc_data.drop(['deadline', 'state_changed_at', 'created_at', 'launched_at'], axis=1, inplace=True)

In [17]:
preproc_data = pd.concat([preproc_data, datecols], axis=1)
preproc_data.columns

Index(['name', 'blurb', 'goal', 'state', 'country', 'creator', 'location',
       'category', 'time_to_state_change', 'pre_launch', 'post_launch'],
      dtype='object')

In [18]:
#preproc_data.isnull().sum() #492 entries with no locations, 5 with no blurb, 1 no name
preproc_data['location'].fillna('No Location', inplace=True)
preproc_data.dropna(axis=0, inplace=True)
preproc_data.drop(['name', 'blurb'], axis=1, inplace=True)

In [63]:
#countries: ['US', 'CA','GB','AU','IE','SE','CH','IT','FR','NZ','DE','NL','NO','MX','ES','DK','AT','BE','HK','SG','LU']

In [64]:
#below attempt to correct the country names for most frequent locations did not improve model's performance, so not using it
def country_fix(df):
    
    df['country'][df['location'].str.contains('Japan')] = 'JP'
    df['country'][df['location'].str.contains('Canada')] = 'CA'
    df['country'][df['location'].str.contains('Italy')] = 'IT'
    df['country'][df['location'].str.contains('France')] = 'FR'
    df['country'][df['location'].str.contains('Mexico')] = 'MX'
    df['country'][df['location'].str.contains('Norway')] = 'NO'
    df['country'][df['location'].str.contains('Spain')] = 'ES'
    df['country'][df['location'].str.contains('Ireland')] = 'IE'
    df['country'][df['location'].str.contains('Sweden')] = 'SE'
    df['country'][df['location'].str.contains('UK')] = 'GB'
    df['country'][df['location'].str.contains('Netherland')] = 'NL'
    df['country'][df['location'].str.contains('Denmark')] = 'DK'
    df['country'][df['location'].str.contains('Austria')] = 'AT'
    df['country'][df['location'].str.contains('Belgium')] = 'BE'
    df['country'][df['location'].str.contains('Germany')] = 'DE'
    df['country'][df['location'].str.contains('Luxembourg')] = 'LU'
    
    return df

#country_fix(preproc_data)

In [65]:
#preproc_data[preproc_data['country'] == 'LU']['location'].unique()
#preproc_data[(preproc_data['location'].astype(str).str.contains('Luxembourg')) & (preproc_data['country'] == 'US')]
#preproc_data[(preproc_data['location'].astype(str).str.contains('Sweden')) & (preproc_data['state'] == 'successful')]
#preproc_data[(preproc_data['location'].astype(str).str.contains('Sweden')) & (preproc_data['country'] == 'US') & (preproc_data['state'] == 'successful')]

In [19]:
preproc_data.to_csv('data\preproc.csv')

In [20]:
pd.set_option('display.max_columns', 14)
preproc_data.head()

,goal,state,country,creator,location,category,time_to_state_change,pre_launch,post_launch
id,,,,,,,,,
663816109,17380.00,failed,US,200609383,"Raleigh, NC",playing cards,1,9 days,30 days
1462931821,24000.00,failed,US,1179655388,"Tokyo, Japan",pop,0,62 days,37 days
1724358498,37069.85,failed,CA,2078972315,"Vancouver, Canada",vegan,0,8 days,30 days
314918941,1000.00,successful,US,863236098,"Albuquerque, NM",mixed media,0,10 days,30 days
1766165140,1121.67,failed,GB,1226281117,"Leicester, UK",children's books,1,4 days,30 days
